In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [3]:
from birdclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark(cores=16, memory="16g")

23/04/14 07:18:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
root = "../data/processed/birdclef-2023/train_embeddings"
! ls {root}
# oops, this is nested too deeply
! ls {root}/embeddings/*/*/*/*.parquet | wc -l

audio  embeddings
6639


In [11]:
df = spark.read.parquet(f"{root}/embeddings/*/*/*/*.parquet")
df.printSchema()
df.count()

root
 |-- embedding: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- predictions: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- track_name: string (nullable = true)



504954

In [13]:
# write this out to a new location
df.repartition(16).write.parquet(f"{root}/consolidated", mode="overwrite")

In [14]:
! ls -alh {root}/consolidated

total 9.3G
drwxr-xr-x 2 anthony anthony 4.0K Apr 14 07:29 .
drwxrwxr-x 5 anthony anthony 4.0K Apr 14 07:28 ..
-rw-r--r-- 1 anthony anthony    8 Apr 14 07:29 ._SUCCESS.crc
-rw-r--r-- 1 anthony anthony 4.6M Apr 14 07:29 .part-00000-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc
-rw-r--r-- 1 anthony anthony 4.6M Apr 14 07:29 .part-00001-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc
-rw-r--r-- 1 anthony anthony 4.6M Apr 14 07:29 .part-00002-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc
-rw-r--r-- 1 anthony anthony 4.6M Apr 14 07:29 .part-00003-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc
-rw-r--r-- 1 anthony anthony 4.6M Apr 14 07:29 .part-00004-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc
-rw-r--r-- 1 anthony anthony 4.6M Apr 14 07:29 .part-00005-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc
-rw-r--r-- 1 anthony anthony 4.6M Apr 14 07:29 .part-00006-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.p

In [15]:
df = spark.read.parquet(f"{root}/consolidated")

In [17]:
df.select("track_name").distinct().count()

39816

In [21]:
df.select("track_name").distinct().orderBy("track_name").show(10, False)
# oops, I made a bunch of duplicates in this dataset

+----------------------------+
|track_name                  |
+----------------------------+
|abethr1/XC128013.mp3        |
|abethr1/XC128013.wav        |
|abethr1/XC128013_source0.mp3|
|abethr1/XC128013_source1.mp3|
|abethr1/XC128013_source2.mp3|
|abethr1/XC128013_source3.mp3|
|abethr1/XC363501.mp3        |
|abethr1/XC363501.wav        |
|abethr1/XC363501_source0.mp3|
|abethr1/XC363501_source1.mp3|
+----------------------------+
only showing top 10 rows



In [33]:
names = (
    df.select(F.split(F.col("track_name"), "_")[0].alias("track_name"))
    .where("track_name not like '%wav' and track_name not like '%mp3'")
    .orderBy("track_name")
    .distinct()
)
names.show()
names.count()

+----------------+
|      track_name|
+----------------+
|combuz1/XC144397|
|cohmar1/XC417422|
|carwoo1/XC435801|
|cohmar1/XC183788|
|blakit1/XC511558|
|eaywag1/XC642065|
|carcha1/XC720297|
|combuz1/XC144232|
|combul2/XC443092|
|afrthr1/XC582597|
| barswa/XC337983|
|eaywag1/XC246326|
|blakit1/XC542603|
|combul2/XC401371|
|combuz1/XC524749|
|colsun2/XC517237|
| egygoo/XC109700|
| barswa/XC337979|
| barswa/XC375299|
|afrthr1/XC466165|
+----------------+
only showing top 20 rows



6636

In [34]:
# upload the dataset to gcs
! gsutil -m rsync -r {root}/consolidated/      gs://birdclef-2023/data/processed/birdclef-2023/train_embeddings_consolidated_v1/

Building synchronization state...
Starting synchronization...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated/.part-00005-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated/.part-00002-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated/.part-00001-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated/.part-00003-701500ef-1a90-4c3b-9ae0-e71e6b2c5f03-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed